In [2]:
import polars as pl
import os

In [8]:
os.chdir("./Notebooks")

# Extract predictions from the intersection DataFrame
* 47 predictions that aren't known Trues
* get the 25 queried ones

In [53]:
first_pred = (
    pl.read_parquet("./data_output/intersection_method_combinations.parquet")
    .filter(pl.col("is_answer") == False, pl.col("method_size") == 7)
    .group_by("h", maintain_order=True)
    .agg(["answers", "filt_rank"])
    .with_columns(
        pl.col("answers").list.first().alias("first_ans"),
        pl.col("filt_rank").list.first().alias("first_rank"),
    )
)

In [54]:
first_pred

h,answers,filt_rank,first_ans,first_rank
str,list[str],list[u32],str,u32
"""CHEBI:6073""","[""DOID:5844""]",[2],"""DOID:5844""",2
"""CHEBI:3286""","[""DOID:14330""]",[1],"""DOID:14330""",1
"""CHEBI:7824""","[""DOID:3312""]",[2],"""DOID:3312""",2
"""CHEBI:28487""","[""DOID:14330""]",[2],"""DOID:14330""",2
"""CHEBI:5556""","[""DOID:14330""]",[3],"""DOID:14330""",3
…,…,…,…,…
"""CHEBI:5051""","[""DOID:2841""]",[1],"""DOID:2841""",1
"""CHEBI:6441""","[""DOID:12849""]",[1],"""DOID:12849""",1
"""CHEBI:9725""","[""DOID:4481""]",[1],"""DOID:4481""",1


### Get names of compound and predicted diseases

In [20]:
nodes = pl.read_csv("../../../MRN_dataset/nodes_biolink.csv").select(
    ["id", "name", "label"]
)

nodes.head(2)

id,name,label
str,str,str
"""UBERON:0000002""","""cervix""","""AnatomicalEntity"""
"""UBERON:0000004""","""human nose""","""AnatomicalEntity"""


In [52]:
# Randomly picked compounds from last curation.

interested = {
    "CHEBI:520985": "almotriptan",
    "CHEBI:78540": "apremilast",
    "CHEBI:77590": "armodafinil",
    "CHEBI:59164": "balsalazide disodium",
    "CHEBI:3286": "cabergoline",
    "CHEBI:3738": "clemastine",
    "CHEBI:3756": "clonazepam",
    "CHEBI:17439": "cyanocob(III)alamin",
    "CHEBI:4046": "cyproheptadine",
    "CHEBI:4638": "diphenidol",
    "CHEBI:4647": "dipivefrin hydrochloride",
    "CHEBI:31530": "edaravone",
    "IKEY:DYLUUSLLRIQKOE-UHFFFAOYSA-N": "Enasidenib",
    "CHEBI:36791": "escitalopram",
    "CHEBI:5051": "Fexofenadine hydrochloride",
    "CHEBI:5139": "Fluvoxamine maleate",
    "CHEBI:6441": "levacetylmethadol",
    "CHEBI:135925": "lisdexamfetamine",
    "CHEBI:31854": "Milnacipran hydrochloride",
    "CHEBI:7575": "nimodipine",
    "CHEBI:8708": "quetiapine fumarate",
    "CHEBI:63620": "rasagiline",
    "CHEBI:9207": "sotalol hydrochloride",
    "CHEBI:9711": "triflupromazine",
    "CHEBI:9725": "Trimeprazine",
}

In [56]:
first_pred.filter(pl.col("h").is_in(interested.keys()))

h,answers,filt_rank,first_ans,first_rank
str,list[str],list[u32],str,u32
"""CHEBI:3286""","[""DOID:14330""]",[1],"""DOID:14330""",1
"""CHEBI:9207""","[""DOID:10763"", ""DOID:2841""]","[1, 2]","""DOID:10763""",1
"""CHEBI:4647""","[""DOID:10763""]",[1],"""DOID:10763""",1
"""CHEBI:4046""","[""DOID:5419""]",[1],"""DOID:5419""",1
"""CHEBI:9711""","[""DOID:5419""]",[1],"""DOID:5419""",1
…,…,…,…,…
"""CHEBI:5051""","[""DOID:2841""]",[1],"""DOID:2841""",1
"""CHEBI:6441""","[""DOID:12849""]",[1],"""DOID:12849""",1
"""CHEBI:9725""","[""DOID:4481""]",[1],"""DOID:4481""",1


In [51]:
nodes.filter(pl.col("name").str.starts_with("Trimeprazine"))

id,name,label
str,str,str
"""CHEBI:9725""","""Trimeprazine""","""ChemicalSubstance"""
